In [ ]:
# prompt: %pip install -Uq upgini catboost

!pip install -Uq upgini catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 46.1 MB

In [ ]:
from posixpath import defpath
from os.path import exists
import pandas as pd

dfpath = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
dataFrame = pd.read_csv(dfpath)
dataFrame = dataFrame.sample(n=19_000, random_state = 0)
dataFrame["store"] = dataFrame["store"].astype(str)
dataFrame["item"] = dataFrame["item"].astype(str)

dataFrame["date"] = pd.to_datetime(dataFrame["date"])
dataFrame.sort_values("date", inplace = True)
dataFrame.head()


,date,store,item,sales
83996,2013-01-01,7,5,5
151558,2013-01-01,4,9,19
584320,2013-01-01,1,33,37
734052,2013-01-01,3,41,14
427284,2013-01-01,5,24,26


In [ ]:
train = dataFrame[dataFrame["date"] < "2017-01-01"]
test = dataFrame[dataFrame["date"] >= "2017-01-01"]


In [ ]:
trainFeature = train.drop(columns = ["sales"])
trainTarget = train["sales"]
testFeature = test.drop(columns = ["sales"])
testTarget = test["sales"]

In [ ]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series



)
enricher.fit(trainFeature,
             trainTarget,
             eval_set=[(testFeature,testTarget)])

[============================================================] 100% Finished

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

f_weather_date_weather_umap_34_c3ef5b4f,0.0497,100.0000,"4.7962, 4.8648, 5.4959",Upgini,Weather & climate normals data,Daily
f_autofe_div_029f75a6,0.0175,100.0000,"0.6054, -0.6231, -0.7663",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_weather_date_weather_pca_0_d7e0a1fc,0.0122,100.0000,"-24.3929, 27.4126, 29.1972",Upgini,Weather & climate normals data,Daily
f_autofe_div_50507976,0.0102,100.0000,"0.062, -0.0187, -0.0344",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_6793ee4f,0.0077,100.0000,"11.4333, 13.8049, 11.0425",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_34d11bc4,0.0064,100.0000,"14.3158, 18.7785, 19.4271",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_1edd5009,0.0061,100.0000,"-0.0434, -0.088, -0.0967",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_50a5dcaf,0.0058,100.0000,"-0.3497, -0.836, -0.9408",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_6234ba3a,0.0055,100.0000,"0.708, 1.0918, 0.8102",Upgini,AutoFE: features from Markets data,Daily
f_autofe_div_3e22df83,0.0042,100.0000,"0.0081, -0.0102, 0.0044",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_89c56a5f,0.0041,100.0000,"-0.8574, -0.7662, 1.155",Upgini,"AutoFE: features from Calendar data,Markets data",Daily


Upgini,"AutoFE: features from Calendar data,Markets data",0.1136,45
Upgini,Weather & climate normals data,0.0674,5
Upgini,AutoFE: features from Markets data,0.0120,11
Upgini,Calendar data,0.0034,2
Upgini,AutoFE: features from Calendar data,0.0008,3
Upgini,Markets data,0.0007,1
Upgini,World economic indicators,0.0002,1


"Calendar data,Markets data",f_autofe_div_029f75a6,f_events_date_week_sin1_847b5db1,f_financial_date_nasdaq_7d_to_7d_1y_shift_4a4dd6ce,/
"Calendar data,Markets data",f_autofe_div_50507976,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,/
"Calendar data,Markets data",f_autofe_mul_6793ee4f,f_events_date_year_cos1_9014a856,f_financial_date_vix_6c8e6194,*
"Calendar data,Markets data",f_autofe_mul_34d11bc4,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,*
"Calendar data,Markets data",f_autofe_div_1edd5009,f_events_date_year_cos1_9014a856,f_financial_date_vix_6c8e6194,/
"Calendar data,Markets data",f_autofe_div_50a5dcaf,f_events_date_week_cos3_7525fe31,f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,/
Markets data,f_autofe_mul_6234ba3a,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,*
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_div_89c56a5f,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_mul_478be4f3,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,*
"Calendar data,Markets data",f_autofe_div_534d01d5,f_events_date_week_sin1_847b5db1,f_financial_date_stoxx_043cbcd4,/



Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15213,50.3977,301.2609,193.9221,107.3388
Eval 1,3787,59.2424,485.3569,362.1067,123.2502


In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
model = CatBoostRegressor(verbose=False, allow_writing_files=False,random_state=0)
enricher.calculate_metrics(
    trainFeature, trainTarget,
    eval_set=[(testFeature,testTarget)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"

)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15213,50.3977,0.252850,0.161732,0.091118
1,Eval 1,3787,59.2424,0.264187,0.199200,0.064987


In [ ]:
enrichedTrainFeatures = enricher.transform(trainFeature,keep_input=True)
enrichedTestFeatures = enricher.transform(testFeature,keep_input=True)
enrichedTestFeatures.head()

[============================================================] 100% Finished

You are trying to launch enrichment for 15213 rows, which will exceed the rest limit 10000.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=c6c2677f-d371-485e-8e44-f4a9a7ac0772
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


,date,store,item,f_weather_date_weather_umap_34_c3ef5b4f,f_autofe_div_029f75a6,f_weather_date_weather_pca_0_d7e0a1fc,f_autofe_div_50507976,f_autofe_mul_6793ee4f,f_autofe_mul_34d11bc4,f_autofe_div_1edd5009,...,f_autofe_div_fb99ff57,f_autofe_mul_c883f8a6,f_events_date_year_sin2_59955ffd,f_autofe_mul_6a97c336,f_autofe_div_0016ede5,f_autofe_mul_44503c6f,f_autofe_mul_8c2f1d42,f_autofe_mul_8437d645,f_autofe_mul_3e749d53,f_autofe_div_ef42018c
786641,2017-01-01,1,44,5.687399,-0.725878,28.311105,0.061824,13.83249,15.700467,0.070172,...,345.226007,389.279864,0.337523,0.501997,37.448371,6.291706,-0.887653,63.83912,-0.97042,0.991476
445179,2017-01-01,4,25,5.687399,-0.725878,28.311105,0.061824,13.83249,15.700467,0.070172,...,345.226007,389.279864,0.337523,0.501997,37.448371,6.291706,-0.887653,63.83912,-0.97042,0.991476
260753,2017-01-01,3,15,5.687399,-0.725878,28.311105,0.061824,13.83249,15.700467,0.070172,...,345.226007,389.279864,0.337523,0.501997,37.448371,6.291706,-0.887653,63.83912,-0.97042,0.991476
910809,2017-01-01,9,50,5.687399,-0.725878,28.311105,0.061824,13.83249,15.700467,0.070172,...,345.226007,389.279864,0.337523,0.501997,37.448371,6.291706,-0.887653,63.83912,-0.97042,0.991476
664299,2017-01-01,4,37,5.687399,-0.725878,28.311105,0.061824,13.83249,15.700467,0.070172,...,345.226007,389.279864,0.337523,0.501997,37.448371,6.291706,-0.887653,63.83912,-0.97042,0.991476


In [ ]:
model.fit(trainFeature, trainTarget)
predictions = model.predict(testFeature)
eval_metric(testTarget.values,predictions,"SMAPE")



[37.65141857448004]

In [ ]:
model.fit(enrichedTrainFeatures, trainTarget)
Enrichedpredictions = model.predict(enrichedTestFeatures)
eval_metric(testTarget.values, Enrichedpredictions, "SMAPE")
print(smape_score)


CatBoostError: X must not be None